In [2]:
import torch.nn as nn
import torch.nn.functional as F


class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(1 * 28 * 28, 256)
        self.drop1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256, 128)
        self.drop2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.drop1(F.leaky_relu(self.fc1(x)))
        x = self.drop2(F.leaky_relu(self.fc2(x)))
        x = F.sigmoid(self.fc3(x))
        return x


class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 1 * 28 * 28)

    def forward(self, x):
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.tanh(self.fc4(x))
        return x

    

In [9]:
# -*- coding: utf-8 -*-
# @Author: aaronlai

import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.utils as vutils
from torch.autograd import Variable


def load_dataset(batch_size=10, download=True):
    """
    The output of torchvision datasets are PILImage images of range [0, 1].
    Transform them to Tensors of normalized range [-1, 1]
    """
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5),
                                                         (0.5, 0.5, 0.5))])
    trainset = torchvision.datasets.MNIST(root='../data', train=True,
                                          download=download,
                                          transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)

    testset = torchvision.datasets.MNIST(root='../data', train=False,
                                         download=download,
                                         transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=2)

    return trainloader, testloader


def gen_noise(n_instance):
    """generate n-dim uniform random noise"""
    return torch.Tensor(np.random.uniform(low=-1.0, high=1.0,
                                          size=(n_instance, 2)))


def train_GAN(Dis_model, Gen_model, D_criterion, G_criterion, D_optimizer,
              G_optimizer, trainloader, n_epoch, batch_size,
              n_update_dis=1, n_update_gen=1, use_gpu=False, print_every=10,
              update_max=None):
    """train GAN and print out the losses for D and G"""
    for epoch in range(n_epoch):

        D_running_loss = 0.0
        G_running_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            # get the inputs from true distribution
            true_inputs, _ = data
            true_inputs = true_inputs.view(-1, 1 * 28 * 28)
            if use_gpu:
                true_inputs = true_inputs.cuda()
            true_inputs = Variable(true_inputs)

            # get the inputs from the generator
            noises = gen_noise(batch_size)
            if use_gpu:
                noises = noises.cuda()
            fake_inputs = Gen_model(Variable(noises))
            print (fake_inputs)
            inputs = torch.cat([true_inputs, fake_inputs])

            # get the labels
            labels = np.zeros(2 * batch_size)
            labels[:batch_size] = 1
            labels = torch.from_numpy(labels.astype(np.float32))
            if use_gpu:
                labels = labels.cuda()
            labels = Variable(labels)

            # Discriminator
            D_optimizer.zero_grad()
            outputs = Dis_model(inputs)
            D_loss = D_criterion(outputs[:, 0], labels)
            if i % n_update_dis == 0:
                D_loss.backward(retain_variables=True)
                D_optimizer.step()

            # Generator
            if i % n_update_gen == 0:
                G_optimizer.zero_grad()
                G_loss = G_criterion(outputs[batch_size:, 0],
                                     labels[:batch_size])
                G_loss.backward()
                G_optimizer.step()

            # print statistics
            D_running_loss += D_loss.data[0]
            G_running_loss += G_loss.data[0]
            if i % print_every == (print_every - 1):
                print('[%d, %5d] D loss: %.3f ; G loss: %.3f' %
                      (epoch+1, i+1, D_running_loss / print_every,
                       G_running_loss / print_every))
                D_running_loss = 0.0
                G_running_loss = 0.0
                vutils.save_image(fake_inputs, 'img.png',nrow=10)
            if update_max and i > update_max:
                break

    print('Finished Training')


def run_GAN(n_epoch=2, batch_size=50, use_gpu=False, dis_lr=1e-4, gen_lr=1e-3,
            n_update_dis=1, n_update_gen=1, update_max=None):
    # loading data
    trainloader, testloader = load_dataset(batch_size=batch_size)

    # initialize models
    Dis_model = Discriminator()
    Gen_model = Generator()

    if use_gpu:
        Dis_model = Dis_model.cuda()
        Gen_model = Gen_model.cuda()

    # assign loss function and optimizer to D and G
    D_criterion = torch.nn.BCELoss()
    D_optimizer = optim.SGD(Dis_model.parameters(), lr=dis_lr, momentum=0.9)

    G_criterion = torch.nn.BCELoss()
    G_optimizer = optim.SGD(Gen_model.parameters(), lr=gen_lr, momentum=0.9)

    train_GAN(Dis_model, Gen_model, D_criterion, G_criterion, D_optimizer,
              G_optimizer, trainloader, n_epoch, batch_size, n_update_dis,
              n_update_gen, update_max=update_max)
    
   
    
if __name__ == '__main__':
    run_GAN()


Variable containing:
 1.3959e-02  3.7743e-02 -7.7520e-02  ...   1.1142e-01  1.6842e-01  2.3392e-02
-5.5088e-03 -1.4488e-01 -1.4694e-01  ...   5.7767e-03  4.2121e-02 -4.3754e-02
 8.1628e-03 -4.2046e-02 -1.2519e-01  ...   4.1316e-02  1.5829e-01 -5.3385e-03
                ...                   ⋱                   ...                
 2.9004e-02 -7.9464e-03 -8.2449e-02  ...   1.1311e-01  1.1480e-01  1.5217e-02
 1.9948e-02  2.2190e-02 -7.6430e-02  ...   1.1922e-01  1.4374e-01  1.7133e-02
 1.4979e-02 -1.4434e-01 -1.2779e-01  ...   5.7635e-02  1.0710e-02 -2.3656e-02
[torch.FloatTensor of size 50x784]

Variable containing:
 6.6612e-03 -2.8107e-02 -1.1285e-01  ...   7.1584e-02  1.5147e-01 -6.9846e-03
 6.1283e-03 -4.8974e-02 -1.2487e-01  ...   4.5427e-02  1.4760e-01 -1.2109e-02
 1.4636e-02  4.2932e-02 -7.5481e-02  ...   1.1245e-01  1.7054e-01  2.6698e-02
                ...                   ⋱                   ...                
 1.4286e-02 -3.1163e-04 -8.5573e-02  ...   1.1229e-01  1.3694e-0

AttributeError: numpy

In [8]:
fake_inputs

NameError: name 'fake_inputs' is not defined

In [5]:
import torch
import numpy as np
array1 = torch.from_numpy(np.array([[1,2,3,4]],dtype=np.float32)).cuda()
array2 = torch.from_numpy(np.array([[2,1,2,3]],dtype=np.float32)).cuda()
(array1 - array2)**2


 1  1  1  1
[torch.cuda.FloatTensor of size 1x4 (GPU 0)]